# Example
## Dummy example use of the FCT model

In [ ]:
import pickle
import numpy as np

from fct.FCT_MIP import FCT_MIP
from fct.FCT_Extended import FCT_Extended
from fct.DataHandler import DataHandler
from fct.UtilityHelper import UtilityHelper
from fct.TreeGenerator import TreeGenerator

In [ ]:
depth = 2
n = 100
p = 5 # 
K = 2 # for non binary is required to slightly change the extending by XGBoost 

X = np.random.rand(n, p)
y = np.random.randint(K, size=n)
dataset_path = "test_data.pickle"

# prepare data in the right format
with open(dataset_path, "wb") as f:
    pickle.dump((X, y, [], [f"feature {i}" for i in range(p)], "dummy_data"), f)

In [ ]:
data_handler = DataHandler(dataset_path)
X_train, y_train = data_handler.get_training_data(split_seed=0, test_size=0.2)
utils = UtilityHelper(data_handler)

fct = FCT_MIP(depth, data_handler, hard_constraint=True)
fct.make_model(X_train, y_train)
fct.optimize(verbose=False, time_limit=200) # setting time limit to 200 seconds

# generates test_tree.pdf and test_tree_red.pdf showing the original and reduced FCT trees respectively
utils.visualize_from_ctx(fct.get_base_context(), "test_tree")

In [ ]:
# Extend the tree with XGBoost models
gen = TreeGenerator(data_handler)
tree = gen.make_from_context(fct.get_base_context())
hybrid = FCT_Extended(tree, data_handler, seed=0, search_iterations=10)

X, y = data_handler.test_data
# this is done, because in normalization we also round to 4 decimals, this puts the data in the expected precision 
X = data_handler.unnormalize(data_handler.normalize(X))
test_leaf_acc, test_acc = hybrid.compute_accuracy(X, y)

print(f"Leaf accuracy is {test_leaf_acc} \nHybrid-tree accuracy is {test_acc}")